In [ ]:
# codigo para testar a dll que nao usa vetores em memoria
import ctypes
import os
from pathlib import Path
import sys
from matplotlib import pyplot as plt
import pandas as pd

class FortranDLLInterface:
    def __init__(self, dll_path: str = 'write_xy_teste.dll'):
        """
        Inicializa a interface com a DLL Fortran.
        
        Args:
            dll_path: Caminho para a DLL (relativo ou absoluto)
        """
        self.dll_path = Path(dll_path).resolve()
        self.fortran_dll = None
        
        try:
            if not self.dll_path.exists():
                raise FileNotFoundError(f"DLL não encontrada em: {self.dll_path}")
            
            # Tenta carregar a DLL
            self.fortran_dll = ctypes.CDLL(str(self.dll_path))
            
            # Configura a função write_xy
            if not hasattr(self.fortran_dll, 'write_xy'):
                raise AttributeError("Função 'write_xy' não encontrada na DLL")
            
            # Define os tipos dos argumentos e retorno
            self.fortran_dll.write_xy.argtypes = [ctypes.c_int]
            self.fortran_dll.write_xy.restype = None
            
        except Exception as e:
            print(f"Erro ao inicializar a DLL: {str(e)}")
            sys.exit(1)
    
    def write_xy(self, n: int) -> bool:
        """
        Chama a função write_xy da DLL Fortran.
        
        Args:
            n: Número de pontos a serem gerados
            
        Returns:
            bool: True se executou com sucesso, False caso contrário
        """
        try:
            if n <= 0:
                raise ValueError("O valor de n deve ser positivo")
            
            print(f"Chamando função Fortran 'write_xy' com n = {n}...")
            self.fortran_dll.write_xy(n)
            
            # Verifica se o arquivo foi gerado
            if not Path('output_xy.csv').exists():
                raise FileNotFoundError("Arquivo 'output_xy.csv' não foi gerado")
            
            print("Função executada com sucesso!")
            return True
            
        except Exception as e:
            print(f"Erro ao executar a função: {str(e)}")
            return False

    # leitura do arquivo gerado csv
    def read_csv(self) -> pd.DataFrame:
        """
        Lê o arquivo 'output_xy.csv' gerado pela função write_xy.
        
        Returns:
            pd.DataFrame: DataFrame com os dados lidos
        """
        try:
            if not Path('output_xy.csv').exists():
                raise FileNotFoundError("Arquivo 'output_xy.csv' não encontrado")
            
            # Lê o arquivo CSV
            df = pd.read_csv('output_xy.csv')
            
            return df
        
        except Exception as e:
            print(f"Erro ao ler o arquivo CSV: {str(e)}")
            return None
    # plotagem do gráfico de x vs y
    def plot(self, df: pd.DataFrame):
        """
        Plota os dados x vs y.
        
        Args:
            df: DataFrame com os dados
        """
        try:
            if df is None:
                raise ValueError("DataFrame não pode ser None")
            
            # Plota o gráfico
            plt.figure(figsize=(10, 6))
            plt.plot(df['x'], df['y'], label='y = sin(x)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.title('Gráfico de y = sin(x)')
            plt.grid(True)
            plt.legend()
            plt.show()
            
        except Exception as e:
            print(f"Erro ao plotar o gráfico: {str(e)}")
def main():
    # Exemplo de uso
    try:
        # Inicializa a interface
        fortran_interface = FortranDLLInterface()
        
        # Chama a função com 1 milhão de pontos
        success = fortran_interface.write_xy(100000)
        
        if success:
            print("Arquivo 'output_xy.csv' gerado com sucesso!")
            # Lê o arquivo gerado
            df = fortran_interface.read_csv()
            # Plota o gráfico
            fortran_interface.plot(df)

        else:
            print("Houve um erro ao gerar o arquivo.")
            
    except Exception as e:
        print(f"Erro inesperado: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main()



In [ ]:
import ctypes
import os
from pathlib import Path
import sys
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

class FortranDLLInterface:
    def __init__(self, dll_path: str = 'write_xy_em_memoria.dll'):
        """
        Inicializa a interface com a DLL Fortran.
        
        Args:
            dll_path: Caminho para a DLL (relativo ou absoluto)
        """
        self.dll_path = Path(dll_path).resolve()
        self.fortran_dll = None
        
        try:
            if not self.dll_path.exists():
                raise FileNotFoundError(f"DLL não encontrada em: {self.dll_path}")
            
            # Carrega a DLL
            self.fortran_dll = ctypes.CDLL(str(self.dll_path))
            
            # Configura a função write_xy_em_memoria
            if not hasattr(self.fortran_dll, 'write_xy_em_memoria'):
                raise AttributeError("Função 'write_xy_em_memoria' não encontrada na DLL")
            
            # Define os tipos dos argumentos e retorno
            self.fortran_dll.write_xy_em_memoria.argtypes = [
                ctypes.c_int,
                np.ctypeslib.ndpointer(dtype=np.float32),
                np.ctypeslib.ndpointer(dtype=np.float32)
            ]
            self.fortran_dll.write_xy_em_memoria.restype = None
            
        except Exception as e:
            print(f"Erro ao inicializar a DLL: {str(e)}")
            sys.exit(1)

    def generate_xy(self, n: int) -> tuple:
        """
        Chama a função write_xy_em_memoria da DLL Fortran.
        
        Args:
            n: Número de pontos a serem gerados
            
        Returns:
            tuple: (x_array, y_array) se sucesso, (None, None) caso contrário
        """
        try:
            if n <= 0:
                raise ValueError("O valor de n deve ser positivo")
            
            # Criar arrays numpy para receber os dados
            x_array = np.zeros(n, dtype=np.float32)
            y_array = np.zeros(n, dtype=np.float32)
            
            print(f"Chamando função Fortran 'write_xy_em_memoria' com n = {n}...")
            self.fortran_dll.write_xy_em_memoria(
                ctypes.c_int(n),
                x_array,
                y_array
            )
            
            print("Função executada com sucesso!")
            return x_array, y_array
            
        except Exception as e:
            print(f"Erro ao executar a função: {str(e)}")
            return None, None

    def plot(self, x_array: np.ndarray, y_array: np.ndarray):
        """
        Plota os dados x vs y.
        """
        try:
            if x_array is None or y_array is None:
                raise ValueError("Arrays não podem ser None")
            
            plt.figure(figsize=(10, 6))
            plt.plot(x_array, y_array, label='y = sin(x)')
            plt.xlabel('x')
            plt.ylabel('y')
            plt.title('Gráfico de y = sin(x)')
            plt.grid(True)
            plt.legend()
            plt.show()
            
        except Exception as e:
            print(f"Erro ao plotar o gráfico: {str(e)}")

def main(n: int = 3000):
    try:
        fortran_interface = FortranDLLInterface()
        x_array, y_array = fortran_interface.generate_xy(n)

        if x_array is not None and y_array is not None:
            print(f"Primeiros 5 valores:")
            print(f"x: {x_array[:5]}")
            print(f"y: {y_array[:5]}")
            print(f"Dimensões: {x_array.shape}, {y_array.shape}")
            
            fortran_interface.plot(x_array, y_array)
        else:
            print("Erro ao gerar os arrays.")
            
    except Exception as e:
        print(f"Erro inesperado: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    main(3000)

#### teste com dll gerada pelo link....

In [ ]:
import ctypes
import os
from pathlib import Path
import sys

class FortranDLLInterface:
    def __init__(self, dll_path: str = './fortran_codes/PortableSquare_mod.dll'):
        self.dll_path = Path(dll_path).resolve()
        
        try:
            if not self.dll_path.exists():
                raise FileNotFoundError(f"DLL not found: {self.dll_path}")
            
            print(f"Loading DLL from: {self.dll_path}")
            self.fortran_dll = ctypes.WinDLL(str(self.dll_path))
            
            # Configure getSq function
            self.square_func = self.fortran_dll.getSq
            self.square_func.argtypes = [ctypes.c_double]
            self.square_func.restype = ctypes.c_double
            
        except Exception as e:
            print(f"DLL initialization error: {str(e)}")
            sys.exit(1)
    
    def calculate_square(self, val: float) -> float:
        try:
            result = self.square_func(ctypes.c_double(val))
            return result
        except Exception as e:
            print(f"Calculation error: {str(e)}")
            return None

def test_interface():
    try:
        interface = FortranDLLInterface()
        test_values = [2.0, 10.0, 100.0]
        
        for val in test_values:
            result = interface.calculate_square(val)
            if result is not None:
                print(f"{val}² = {result}")
                
    except Exception as e:
        print(f"Test error: {str(e)}")
        sys.exit(1)

if __name__ == "__main__":
    test_interface()

In [3]:
import ctypes
from pathlib import Path
import sys

class HelloWorldDLL:
    def __init__(self, dll_path: str = r'C:\Users\Miguel\source\repos\Hello_world_dll\x64\Debug\Hello_world_dll.dll'):
        """Initialize the DLL interface"""
        self.dll_path = Path(dll_path)
        self.dll = None
        
        try:
            if not self.dll_path.exists():
                raise FileNotFoundError(f"DLL not found: {self.dll_path}")
            
            print(f"Loading DLL from: {self.dll_path}")
            
            # Load DLL
            self.dll = ctypes.WinDLL(str(self.dll_path))
            
            # Get the HELLO_WORLD function
            if not hasattr(self.dll, 'HELLO_WORLD'):
                raise AttributeError("Function 'HELLO_WORLD' not found in DLL")
            
            # Configure function signature
            self.hello_func = self.dll.main
            self.hello_func.argtypes = []  # No arguments
            self.hello_func.restype = None  # Void return type
            self.hello_func.stdout = sys.stdout  # Redirect stdout to console

            print("DLL loaded successfully")
            
        except Exception as e:
            print(f"DLL initialization error: {e}")
            sys.exit(1)
    
    def say_hello(self):
        """Call the HELLO_WORLD function"""
        try:
            print("\nCalling HELLO_WORLD function...")
            self.hello_func()
            return True
        except Exception as e:
            print(f"Function call error: {e}")
            return False

def test():
    try:
        # Create interface instance
        dll = HelloWorldDLL()
        
        # Call the function
        result = dll.say_hello()
        
        # Print result
        print("\nTest", "succeeded" if result else "failed")
        
    except Exception as e:
        print(f"Test error: {e}")
        sys.exit(1)

if __name__ == "__main__":
    test()

Loading DLL from: C:\Users\Miguel\source\repos\Hello_world_dll\x64\Debug\Hello_world_dll.dll
DLL loaded successfully

Calling HELLO_WORLD function...

Test succeeded


In [1]:
import ctypes
import os
from pathlib import Path

class FortranOutput:
    def __init__(self):
        self.output_buffer = ctypes.create_string_buffer(1024)
        
class DLLInterface:
    def __init__(self, dll_path: str):
        os.environ['FOR_DISABLE_CONSOLE_CTRL_HANDLER'] = '1'
        self.dll = ctypes.WinDLL(str(Path(dll_path)))
        self.output = FortranOutput()
        
        # Configure main function
        self.main_func = self.dll.main
        self.main_func.argtypes = []
        self.main_func.restype = None
        
        print("DLL loaded successfully")

    def call_main(self):
        print("\nCalling Fortran main function...")
        # Redirect stdout to a pipe
        import subprocess
        
        # Run DLL function with output capture
        process = subprocess.Popen(['python', '-c', 
            'import ctypes; dll=ctypes.WinDLL("' + str(self.dll._name) + '"); dll.main()'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True)
        
        stdout, stderr = process.communicate()
        print("Captured Fortran output:")
        print(stdout)
        return stdout

# Test execution
dll_path = r"C:\Users\Miguel\source\repos\Hello_world_dll\x64\Debug\Hello_world_dll.dll"
dll = DLLInterface(dll_path)
result = dll.call_main()

DLL loaded successfully

Calling Fortran main function...
Captured Fortran output:

